PostId: 2018-08-18-221456
Title:新ブログシステムを考える

### あらすじ

現行のブログシステムは、間にgithub issueを挟む。
これはPCから編集出来るように、とか、アプリ側に違う物も使えるように、とかの理由でこうしたのだが、

1. PCなんてもうブログ編集には使わない
2. ブログ書くアプリはMeatPieDay以外は無い

という事で、あまりメリットが無い気がしてきている。

さらに間にissueに挟む副作用として

1. 画像が外部サービス（imgur）
   - 一枚ずつ一リクエストで送ってるのでアップロードが遅い
   - 外部サービスは終了しちゃうと困る
2. 追加にしか対応してないので、誤植を見つけた時に直すのがかったるい（issue上で直せば良いが、手元のファイルと不整合が起こる）
3. issueとgithubとimgurという3つのサービスに依存してるので、何かトラブルが起きた時にトラブルシュートがかったるい

などの面倒さが出てきた。

特に先週くらいから、なんかTravisからgithub issueへのGETが500を返すようになって動かず、原因追求がかったるい（ローカルでは動いてる）。

幸い、電子書籍をMeatPieDayで書く環境を作った時に、個々のパーツとしてはブログでも使えそうな物を一通り書いたので、この機会にブログシステムを替えたい、と思った。

### 旧構成

メモとして残しておく。

1. MeatPieDayで書く
2. mpd2issueでissueに登録
3. IncrementCounterというアプリでbuildブランチのcounter.txtを更新してコミット
4. CIがgithub-issues2markdownというgoで書いたコマンドを叩き、これがissueをなめてmarkdownを生成しコミット

でjekyllがブログにしていた訳だ。

なお、2の所でissueを建てる時に、issueのidをnotificationで表示していた。
これをタップするとクリップボードにidが入って、これを最初のセルにコピペしておくと、以後2はそのissueへの更新、とみなすようになる。

だいたいのブログは書き捨てなのでこのidを貼る、という作業はいらないが、継続的に更新する奴はこの作業が要る。
この必要な奴だけidを埋める、というのは崩したくない。つまり書き捨ての都度id入れるのは嫌だ。

この構成のメリットとしては、投稿時は差分しかポストしないので、長いエントリの時に通信量が少ない事。

だが、長いエントリでもせいぜい3MB程度なので、イマドキなら全部送れば良いかな、という気がする。
なお、これまでで最大のエントリはdudley本のReal Analysisで2.8MB。あれで2.8MBなら5MBに行く事はほぼなかろう。

## 新構成

1. MaatPieDayで書く（旧版と一緒）
2. mpd2blogでgithubの特定のブランチにipynbを置く
3. CIでipynbからマークダウンと画像を生成してcommit（ipynb2markdownというgoコマンドを作ろう）

という構成にしたい。2と3は、似たような事をするのをebook生成の時書いたので、それを流用したいが、ちょっと変える必要がある。dupしていじるかちゃんと両対応にするかは悩ましい。

### 新構成のメリット

1. ipynbをそのまま上げるので、バックアップにもなってる。
2. 複数端末から更新するのが比較的容易。
3. 毎回ファイルごと上げ直しなので、途中のセルの変更も簡単
4. githubとtravisにしか依存しないので、トラブルが減る（とイイナ！）

使っている身としては3が大きい。
誤植直すのが手軽に出来るようになるので。

あとバックアップになってるのも安心感がある。
旧構成でもissueとimgurからipynbは再構成出来るはずなのだが、スクリプトを書かないと出来ないし、本当にやるかも分からない（サービス終了時などに、めんどくなっていいや、となるのはいかにもありそう）

という事で、かなり良くなるんじゃないか。

## mpd2blog

ipynbをgithubにcommitするアプリだが、ID回りでちょこちょこ作業がある。
特にIDの入ったセルを追加するのがかったるいかもしれない。

という事でまずはIDの仕様について考える。

### PostIDの仕様

これまではissueをたてる時に自動的にidが振られたが、今度は自分で決めないといけない。
サーバー側にcounter.txtとかで現在の最後のidを持っておいてもいいが、二回ラウンドトリップが発生するのでかったるい。

日付とtimestampでいい気もする。どうせかぶらないし。

書き捨てのブログで最初にidを貼るのはかったるいので、idが無い時はその時のtimestampから生成したいが、ファイル名とかどうしよう？

github上で日本語ファイル名はいろいろトラブルありそうなので、このidをファイル名にしたい。
一方で元のMeatPieDayはファイル名をタイトルとしていて、こっちは変えたくない。

すると、以下のようになるか。

**最初のセルにPostIDという値が無い時**

1. mpd2blogは、タイムスタンプからIDを生成し、それを最初のセルにPostIDという名前で追加
2. sendで送られてきたファイル名をタイトルとして、Titleという名前で追加
3. postするファイル名はPostIDのファイル名（元と違う）
4. ポスト後にnotificationで作ったPostIDを表示する

最初のセルにPostIDが無ければ、メタデータ用のセルが最初に無いと思って自動で追加する。だからTitleだけのセル、とかでは認識しない事にする。

**最初のセルにPostIDという値がある時**

1. githubにはPostIDのファイル名にポストする
2. Titleは毎回その時にSendで渡ってくるファイル名で上書きする

後日変更したくなった時、ipynbからインポートすると、この仕様ではタイトルは手作業でインポート後に直さないといけないが、タイトル変更したくなった時に簡単に変えられる方が良かろう、という事でこの仕様で。